In [1]:
!pip install tensorflow

  Ignoring futures: markers 'python_version < "3.2"' don't match your environment
  Ignoring wheel: markers 'python_version < "3"' don't match your environment
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


O objetivo deste trabalho é comparar diversos métodos de classificação para a base de dados de qualidade de vinhos disponível em https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv.

Vocês devem encontrar um bom modelo preditivo, variando:
* o número e conjunto de features (atributos) utilizados
* o método utilizado
* a configuração do algoritmo correspondente (e.g.: número k para nearest neighbors, profundidade para árvore de decisão)

Vocês devem listar algumas métricas de qualidade, tais como: precision, recall, accuracy e f1_score, e utilizar accuracy como base para a avaliação final, considerando a accuracy média de 10 iterações para cada configuração.

Para assegurar que eu obterei os mesmos resultados de vocês, vocês devem estabelecer a semente para a geração dos números aleatórios (utilizados para separar os conjuntos de treinamento e teste, por exemplo), utilizando os seguintes comandos no início do seu código (podem utilizar uma outra semente):
```
import random
random.seed(1001001)
```

In [28]:
# Rafael Augusto Gaseta França - 1421436
# Pedro Ferreira Alves Pinto - 1320981

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

%matplotlib inline

In [29]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv' ,sep=';')

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [30]:
df.quality.describe()

count    1599.000000
mean        5.636023
std         0.807569
min         3.000000
25%         5.000000
50%         6.000000
75%         6.000000
max         8.000000
Name: quality, dtype: float64

In [31]:
df.quality.unique()

array([5, 6, 7, 4, 8, 3])

In [32]:
X = df.iloc[:, :-1].values
Y = df.iloc[:,-1].values

import random
random.seed(1001001)

from sklearn.preprocessing import normalize

X = normalize(X)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

### Linear Regression

In [33]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [34]:
y_pred = regressor.predict(X_test)

y_round = []
for y in y_pred:
    y_round.append(int(round(y)))

In [35]:
np.mean(Y_test == y_round)

0.63125

### SVM

In [36]:
from sklearn.svm import SVC

for i in range(1, 50, 2):
    svm = SVC(C = i, kernel = 'linear')
    svm.fit(X_train, Y_train)
    print('C={} -> {}'.format(i, svm.score(X_test, Y_test)))

C=1 -> 0.534375
C=3 -> 0.55
C=5 -> 0.55625
C=7 -> 0.553125
C=9 -> 0.565625
C=11 -> 0.56875
C=13 -> 0.58125
C=15 -> 0.58125
C=17 -> 0.5875
C=19 -> 0.59375
C=21 -> 0.6
C=23 -> 0.6
C=25 -> 0.609375
C=27 -> 0.6125
C=29 -> 0.6125
C=31 -> 0.60625
C=33 -> 0.60625
C=35 -> 0.60625
C=37 -> 0.60625
C=39 -> 0.60625
C=41 -> 0.60625
C=43 -> 0.609375
C=45 -> 0.609375
C=47 -> 0.615625
C=49 -> 0.615625


In [37]:
# aparentemente quanto maior o C maior o resultado

svm = SVC(C = 100, kernel = 'linear')
svm.fit(X_train, Y_train)
print(svm.score(X_test, Y_test))
# entretanto aparenta chegar a um limite

0.615625


### K-NN

In [38]:
from sklearn.neighbors import KNeighborsClassifier

for i in range(1, 50, 2):
    knn = KNeighborsClassifier(n_neighbors = i).fit(X_train, Y_train)
    print('n_neighbors={} -> {}'.format(i, knn.score(X_test, Y_test)))

n_neighbors=1 -> 0.603125
n_neighbors=3 -> 0.521875
n_neighbors=5 -> 0.475
n_neighbors=7 -> 0.478125
n_neighbors=9 -> 0.503125
n_neighbors=11 -> 0.49375
n_neighbors=13 -> 0.503125
n_neighbors=15 -> 0.503125
n_neighbors=17 -> 0.509375
n_neighbors=19 -> 0.521875
n_neighbors=21 -> 0.525
n_neighbors=23 -> 0.509375
n_neighbors=25 -> 0.528125
n_neighbors=27 -> 0.534375
n_neighbors=29 -> 0.51875
n_neighbors=31 -> 0.53125
n_neighbors=33 -> 0.534375
n_neighbors=35 -> 0.53125
n_neighbors=37 -> 0.528125
n_neighbors=39 -> 0.5375
n_neighbors=41 -> 0.55
n_neighbors=43 -> 0.540625
n_neighbors=45 -> 0.528125
n_neighbors=47 -> 0.54375
n_neighbors=49 -> 0.534375


### Multi Layer Perceptron

In [39]:
from sklearn.neural_network import MLPClassifier

for i in range(1,5):
    for j in range(1,5):
        clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(i, j), random_state=60)
        clf.fit(X_train, Y_train)   
        print('hidden_layer_sizes={} -> {}'.format((i, j), clf.score(X_test, Y_test)))

hidden_layer_sizes=(1, 1) -> 0.63125
hidden_layer_sizes=(1, 2) -> 0.640625
hidden_layer_sizes=(1, 3) -> 0.421875
hidden_layer_sizes=(1, 4) -> 0.65
hidden_layer_sizes=(2, 1) -> 0.63125
hidden_layer_sizes=(2, 2) -> 0.634375
hidden_layer_sizes=(2, 3) -> 0.609375
hidden_layer_sizes=(2, 4) -> 0.621875
hidden_layer_sizes=(3, 1) -> 0.421875
hidden_layer_sizes=(3, 2) -> 0.65
hidden_layer_sizes=(3, 3) -> 0.421875
hidden_layer_sizes=(3, 4) -> 0.61875
hidden_layer_sizes=(4, 1) -> 0.625
hidden_layer_sizes=(4, 2) -> 0.634375
hidden_layer_sizes=(4, 3) -> 0.625
hidden_layer_sizes=(4, 4) -> 0.6125


OS resultados foram bem similares, tirando o caso de (1,3) e (3,3)

### Decision Tree

In [40]:
from sklearn.tree import DecisionTreeClassifier


dt = DecisionTreeClassifier()
dt.fit(X_train, Y_train)
dt.score(X_test, Y_test)

0.603125

### Neural Network

In [41]:
D = X.shape[1] # input dim
M = 11 # hidden size
K = 6 # number classes

In [42]:
T = np.zeros((Y.size, K))
for i in range(Y.size):
    T[i, Y[i]-3] = 1

T_train = np.zeros((Y_train.size, K))
for i in range(Y_train.size):
    T_train[i, Y_train[i]-3] = 1
    
T_test = np.zeros((Y_test.size, K))
for i in range(Y_test.size):
    T_test[i, Y_test[i]-3] = 1

In [43]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.1))

W1 = init_weights([D, M])
b1 = init_weights([M])

W2 = init_weights([M, M])
b2 = init_weights([M])

W3 = init_weights([M, M])
b3 = init_weights([M])

Wf = init_weights([M, K])
bf = init_weights([K])

In [44]:
tfX = tf.placeholder(tf.float32, [None, D])
tfY = tf.placeholder(tf.float32, [None, K])

# MultiLayer Perceptron
def forward():
    Z = tf.nn.sigmoid(tf.matmul(tfX, W1) + b1)
    Z2 = tf.nn.sigmoid(tf.matmul(Z, W2) + b2)
    Z3 = tf.nn.sigmoid(tf.matmul(Z2, W3) + b3)
    
    return tf.matmul(Z3, Wf) + bf

py_x = forward()

# Treinado com Softmax para um output multiclasse
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tfY, logits=py_x))

In [45]:
train_op = tf.train.AdamOptimizer().minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [25]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

from tqdm import tqdm
pbar= tqdm(range(15000))

for i in pbar:
    sess.run(train_op, feed_dict={tfX: X_train, tfY: T_train})
    pred_train = sess.run(predict_op, feed_dict={tfX: X_train, tfY: T_train})
    pred_test = sess.run(predict_op, feed_dict={tfX: X_test, tfY: T_test})
    
    if i%1000 == 0:
        pbar.set_description("acc train: {}; acc test: {}".format(np.mean(Y_train-3 == pred_train), 
                                                                  np.mean(Y_test-3 == pred_test)))

acc train: 0.49335418295543393; acc test: 0.528125:  22%|██▏       | 3274/15000 [00:09<00:35, 327.51it/s]/home/pedalv/miniconda3/envs/jupyter/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
acc train: 0.6411258795934324; acc test: 0.64375: 100%|██████████| 15000/15000 [00:45<00:00, 326.56it/s] 


O rsultado do mlp feito com o TensowFlow apresenta resultados similares ao do SKLearn, o que faz refletir a real necessidade de se desenvolver um algoritimo "basico" no lugar de usar um ja desenvolvido 